In [ ]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

api_key = '4x84FUc-7Tiy-ATM2Fey9egqFD-dOjLk5vStKusogaLS7X3bZMAaqHrNTdSMpTFA44FejG97zczOlCRrgLMHSqVeZ0MmABv7bqtsvMmHcUTKZPqpZNBl-6qQkceDXXYx'

# headers
header = {'Authorization':'bearer {}'.format(api_key),
         'Content-Type': 'application/json'}

# request network
transport = RequestsHTTPTransport(url = 'https://api.yelp.com/v3/graphql', headers = header, use_json = True)

# client
client = Client(transport = transport, fetch_schema_from_transport = True)


In [ ]:
def format_query(limit, offset):
    query = gql('''
    {
      search(term: "",
         location: "long beach",
  		 limit: ''' + str(limit) + ''',
         offset: ''' + str(offset) + ''' ) {
            total
            business {
              id
              name
              review_count
              photos
              url
              categories {
                title
                alias
              }
              coordinates {
                latitude
                longitude
              }
              location {
                address1
                city
                state
                country
              }
            }
        }
    }

    ''')
    return query
    

In [ ]:
def get_line(input):
    line_composition = []
    
    line_composition.append(input['id'])
    line_composition.append(input['name'])
    line_composition.append(format_address(input))
    line_composition.append(input['coordinates']['longitude'])
    line_composition.append(input['coordinates']['latitude'])
    line_composition.append(input['url'])
    line_composition.append(input['photos'][0])
    line_composition.append(format_category(input))
    line_composition.append(input['review_count'])
       
    return line_composition

def format_address(input):
    address = ''
    
    for line in input['location']:
        address += input['location'][line] + ' '
    
    return address

def format_category(input):
    category = ''
    
    for line in input['categories']:
        category += (line['title']) + ' '
        
    return category

In [ ]:
import csv

offset = 0
limit = 50

while True:
    response_query = client.execute(format_query(limit, offset))
    
    for x in response_query['search']['business']:
    
        with open(r'yelp_api_data.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow(get_line(x)) 
        
    if int(response_query['search']['total']) < offset:
        break
        
    offset += 50
    